# Drowsiness Detection Notebook
This Jupyter notebook replicates the GUI functionality described earlier. It allows you to:

* Detect multiple people in images or video streams using **YOLOv8**.
* Determine whether each detected face is **awake (green box)** or **sleeping (red box)** via eye–aspect ratio (EAR).
* Display how many people are sleeping along with (placeholder) age estimates.

The notebook is CPU-friendly but will use a GPU automatically if one is available.


In [3]:

from ultralytics import YOLO
import cv2, dlib, numpy as np, math, random, os, urllib.request
from imutils import face_utils
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets


In [4]:

# Download YOLOv8n weights if not present
if not os.path.exists('yolov8n.pt'):
    print('Downloading YOLOv8n weights…')
    urllib.request.urlretrieve('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt', 'yolov8n.pt')

# Download dlib face-landmark model if not present
if not os.path.exists('shape_predictor_68_face_landmarks.dat'):
    print('Downloading Dlib facial landmark model (99 MB)…')
    urllib.request.urlretrieve(
        'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2',
        'shape_predictor_68_face_landmarks.dat.bz2')
    import bz2, shutil
    with bz2.open('shape_predictor_68_face_landmarks.dat.bz2', 'rb') as f_in, open('shape_predictor_68_face_landmarks.dat', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    os.remove('shape_predictor_68_face_landmarks.dat.bz2')


In [5]:

# Load YOLOv8 for person detection
model = YOLO('yolov8n.pt')

# Initialize dlib face detector & predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


In [6]:

import numpy as np, math, random

def eye_aspect_ratio(eye):
    A = math.dist(eye[1], eye[5])
    B = math.dist(eye[2], eye[4])
    C = math.dist(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_drowsiness(frame, ear_threshold=0.25):
    """Detect persons and classify each as sleeping/awake with placeholder age."""
    results = model.predict(source=frame, classes=[0], conf=0.25, verbose=False)[0]
    detections = []
    for box in results.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = box.astype(int)
        person_crop = frame[y1:y2, x1:x2]
        gray = cv2.cvtColor(person_crop, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 0)
        status = 'Awake'
        age = None
        for face in faces:
            shape = landmark_predictor(gray, face)
            shape = face_utils.shape_to_np(shape)
            (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
            (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            ear = (eye_aspect_ratio(leftEye) + eye_aspect_ratio(rightEye)) / 2.0
            status = 'Sleeping' if ear < ear_threshold else 'Awake'
            age = random.randint(18, 60)  # placeholder
            break
        detections.append({'bbox': (x1, y1, x2, y2), 'status': status, 'age': age})
    return detections

def annotate_frame(frame, detections):
    sleeping_count = 0
    ages = []
    for det in detections:
        x1, y1, x2, y2 = det['bbox']
        color = (0, 0, 255) if det['status'] == 'Sleeping' else (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"{det['status']}" + (f", {det['age']}y" if det['age'] else '')
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        if det['status'] == 'Sleeping':
            sleeping_count += 1
            if det['age']:
                ages.append(det['age'])
    return frame, sleeping_count, ages


In [7]:

#@title Upload and test on an image { run: "auto" }
from IPython.display import clear_output, Image as IPImage
uploader = widgets.FileUpload(accept='image/*', multiple=False)
print('Upload an image:')

display(uploader)

def on_upload_change(change):
    if uploader.value:
        content = next(iter(uploader.value.values()))['content']
        img_array = np.frombuffer(content, dtype=np.uint8)
        frame = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        detections = detect_drowsiness(frame)
        annotated, sleeping, ages = annotate_frame(frame.copy(), detections)
        annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        clear_output(wait=True)
        print(f'Sleeping people: {sleeping}', end='')
        if ages:
            print(' - ages:', ', '.join(map(str, ages)))
        else:
            print()
        display(IPImage(data=cv2.imencode('.png', annotated)[1].tobytes()))

a = uploader.observe(on_upload_change, names='value')


Upload an image:


FileUpload(value=(), accept='image/*', description='Upload')

In [8]:

# Demo processing a short video file (change path as needed)
video_path = 'sample_video.mp4'  # replace with your file or leave blank for webcam
if not os.path.exists(video_path):
    print('Provide a valid video file path in video_path variable above.')
else:
    cap = cv2.VideoCapture(video_path)
    out_widget = widgets.Image()
    display(out_widget)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        detections = detect_drowsiness(frame)
        annotated, sleeping, ages = annotate_frame(frame.copy(), detections)
        annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        _, png = cv2.imencode('.png', annotated)
        out_widget.value = png.tobytes()
    cap.release()


Provide a valid video file path in video_path variable above.
